In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources

from tensorflow.examples.tutorials.mnist import input_data

In [14]:
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

#set parameters
epoches = 500
batch_size = 1024
num_classes = 10
num_features = 784
num_trees = 10
max_nodes = 1000

tf.reset_default_graph()
#input and target data
X = tf.placeholder(shape=[None, 784], dtype=tf.float32, name='train_x')
Y = tf.placeholder(shape=[None], dtype=tf.int32, name='label')


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [15]:
hparams = tensor_forest.ForestHParams(num_trees=num_trees,
                    num_features=num_features,
                    num_classes=num_classes,
                    max_nodes=max_nodes).fill()

forest_graph = tensor_forest.RandomForestGraphs(hparams)

train_op = forest_graph.training_graph(X, Y)
loss_op = forest_graph.training_loss(X, Y)
#返回森林中每棵树对于X投票结果（第i类：得票数）
inference_op, _, _ = forest_graph.inference_graph(X)
correct_prediction = tf.equal(tf.arg_max(inference_op, 1), tf.cast(Y, tf.int64))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init_vars = tf.group(tf.global_variables_initializer(),
                  resources.initialize_resources(resources.shared_resources()))

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_trees': 10, 'max_nodes': 1000, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'num_splits_to_consider': 28, 'max_fertile_nodes': 0, 'split_after_samples': 250, 'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'dominate_fraction': 0.99, 'model_name': 'all_dense', 'split_finish_name': 'basic', 'split_pruning_name': 'none', 'collate_examples': False, 'checkpoint_stats': False, 'use_running_stats_method': False, 'initialize_average_splits': False, 'inference_tree_paths': False, 'param_file': None, 'split_name': 'less_or_equal', 'early_finish_check_every_samples': 0, 'prune_every_samples': 0, 'num_features': 784, 'num_classes': 10, 'bagged_num_features': 784, 'bagged_features': None, 'regression': False, 'num_outputs': 1, 'num_output_columns': 11, 'base_random_seed': 0, 'leaf_model_type': 0, 'stats_model_type': 0, 'finish_type': 0, 'pruning_type': 0, 'split_type': 0}


In [20]:
sess = tf.train.MonitoredSession()

sess.run(init_vars)

for i in range(1, epoches+1):
  batch_x, batch_y = mnist.train.next_batch(batch_size)
  batch_y = np.argmax(batch_y, 1)
  _, l = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y:batch_y})
  if i%50==0:
    acc = sess.run(accuracy_op, feed_dict={X: batch_x, Y:batch_y})
    print('Step  % i, Loss: %f, Acc: %f' % (i, l, acc))

# Test Model
test_x, test_y = mnist.test.images, mnist.test.labels
test_y = np.argmax(test_y, 1)
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Step   50, Loss: -255.000000, Acc: 0.877930
Step   100, Loss: -540.000000, Acc: 0.900391
Step   150, Loss: -832.799988, Acc: 0.926758
Step   200, Loss: -1001.000000, Acc: 0.920898
Step   250, Loss: -1001.000000, Acc: 0.929688
Step   300, Loss: -1001.000000, Acc: 0.916016
Step   350, Loss: -1001.000000, Acc: 0.943359
Step   400, Loss: -1001.000000, Acc: 0.926758
Step   450, Loss: -1001.000000, Acc: 0.919922
Step   500, Loss: -1001.000000, Acc: 0.933594
Test Accuracy: 0.9218
